In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import random_clifford, Pauli, Statevector
from qiskit_aer import AerSimulator, Aer
from IPython.display import clear_output, display
from time import sleep

# Function to apply a random Clifford transformation
def apply_random_clifford(qc, qubits):
    clifford = random_clifford(len(qubits))
    qc.append(clifford, qubits)
    return clifford

# Function to measure in the computational basis
def measure_computational(qc, qubits):
    qc.measure(range(qubits), range(qubits))

# Function to reconstruct the density matrix from classical shadows
def reconstruct_density_matrix(shadow, clifford, num_qubits):
    dim = 2 ** num_qubits
    U_inv = clifford.to_matrix().conj().T
    rho = (dim + 1) * (U_inv @ np.outer(shadow, shadow) @ U_inv.conj().T) - np.eye(dim)
    return rho

# Main simulation
num_qubits = 2
num_shadows = 10000
expectations, temp_expectations = {}, {}
pauli_observables = ['XX', 'YY', 'ZZ'] # List of observables
for obs in pauli_observables:
    expectations[obs] = []
    temp_expectations[obs] = []

# Initialize quantum circuit
def prepare_qc(n):
    return QuantumCircuit(n, n)

# Apply random Clifford transformations and measure
simulator = Aer.get_backend('qasm_simulator')
for i in range(num_shadows):
    qc = prepare_qc(num_qubits)
    clifford = apply_random_clifford(qc, range(num_qubits)) # Apply Clifford gates
    measure_computational(qc, num_qubits) # Measure in computational basis
    compiled_circuit = transpile(qc, simulator)
    qobj = assemble(compiled_circuit, shots=1) # One shot for each Clifford gate
    result = simulator.run(compiled_circuit, shots=1).result()
    counts = result.get_counts()
    vector = np.zeros(2**num_qubits) 
    index = int(counts.popitem()[0], 2) # Access the measured result and convert to vector
    vector[index] = 1
    rho = reconstruct_density_matrix(vector, clifford, num_qubits)
    for obs in pauli_observables:
        temp_expectations[obs].append(np.trace(rho @ Pauli(obs).to_matrix()))
        if i > 0 and i % 25 == 0: # Apply median-of-means post-processing
            expectations[obs].append(np.mean(temp_expectations[obs]))
            temp_expectations[obs] = []
    clear_output()
    display(f'task {i+1} / {num_shadows} done')

# Print results
print("\nExpectation values of Pauli observables:")
for pauli, expectation in expectations.items():
    print(f"{pauli}: {np.median(expectation)}")

'task 10000 / 10000 done'


Expectation values of Pauli observables:
XX: 0j
YY: 0j
ZZ: (0.9999999999999994+0j)
